## Slicing 2D Array

In [1]:
import numpy as np

In [2]:
b = np.array([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]])

In [3]:
print(b)

[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]]


In [4]:
b[:, 1]  # 첫번째 차원 (행)은 모두, 두번째 차원 (열)은 인덱스 1값만!

array([ 2,  6, 10])

In [5]:
b[-1]   # 첫번째 차원의 맨 마지막 값 (마지막 행의 원소들)

array([ 9, 10, 11, 12])

In [6]:
b[-1, :] # 첫번째 차원의 맨 마지막 값 (마지막 행의 원소들), 두번째 차원은 모두

array([ 9, 10, 11, 12])

In [7]:
b[-1, ...]  # 첫번째 차원의 맨 마지막 값 (마지막 행의 원소들), 두번째 차원은 모두

array([ 9, 10, 11, 12])

In [8]:
b[0:2, :]  # 첫번째 차원(행)은 index 0,1 값만 가져오기, 두번째 차원(열)은 모두 가져오기

array([[1, 2, 3, 4],
       [5, 6, 7, 8]])

## Loading Data from `.csv` file

In [9]:
xy = np.loadtxt('data-01-test-score.csv', delimiter=',', dtype=np.float32)

In [10]:
x_data = xy[:, 0:-1]   #첫번째 차원(행)은 모두, 두번째 차원(열)은 마지막 인덱스 제외
y_data = xy[:, [-1]]   #첫번째 차원(행)은 모두, 두번째 차원(열)은 마지막 인덱스만!

In [11]:
print(x_data.shape) # x_data shape
print(len(x_data))  # x_data 길이
print(x_data[:5])   # 첫 다섯 개

(25, 3)
25
[[ 73.  80.  75.]
 [ 93.  88.  93.]
 [ 89.  91.  90.]
 [ 96.  98. 100.]
 [ 73.  66.  70.]]


In [12]:
print(y_data.shape) # y_data shape
print(len(y_data))  # y_data 길이
print(y_data[:5])   # 첫 다섯 개

(25, 1)
25
[[152.]
 [185.]
 [180.]
 [196.]
 [142.]]


## Import Required Libraries

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [14]:
# For reproducibility
torch.manual_seed(1)

## Low-level Implementation

모듈 클래스 없이 구현하기

In [15]:
x_train = torch.FloatTensor(x_data)    # 데이터를 Tensor로 받기
y_train = torch.FloatTensor(y_data)

# 모델 초기화
W = torch.zeros((3, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    hypothesis = x_train.matmul(W) + b # or .mm or @

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

Epoch    0/20 Cost: 26811.960938
Epoch    1/20 Cost: 9920.530273
Epoch    2/20 Cost: 3675.299072
Epoch    3/20 Cost: 1366.260620
Epoch    4/20 Cost: 512.542480
Epoch    5/20 Cost: 196.896606
Epoch    6/20 Cost: 80.190994
Epoch    7/20 Cost: 37.038696
Epoch    8/20 Cost: 21.081335
Epoch    9/20 Cost: 15.178759
Epoch   10/20 Cost: 12.993677
Epoch   11/20 Cost: 12.183028
Epoch   12/20 Cost: 11.880536
Epoch   13/20 Cost: 11.765964
Epoch   14/20 Cost: 11.720855
Epoch   15/20 Cost: 11.701437
Epoch   16/20 Cost: 11.691509
Epoch   17/20 Cost: 11.685122
Epoch   18/20 Cost: 11.680006
Epoch   19/20 Cost: 11.675371
Epoch   20/20 Cost: 11.670942


## High-level Implementation with `nn.Module`

In [16]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)

    def forward(self, x):
        return self.linear(x)

In [17]:
# 데이터
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

# 모델 초기화
model = MultivariateLinearRegressionModel()

# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs+1):
    
    # H(x) 계산
    prediction = model(x_train)
    
    # cost 계산
    cost = F.mse_loss(prediction, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 20번마다 로그 출력
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

Epoch    0/20 Cost: 28693.488281
Epoch    1/20 Cost: 10618.750000
Epoch    2/20 Cost: 3936.015381
Epoch    3/20 Cost: 1465.219604
Epoch    4/20 Cost: 551.693726
Epoch    5/20 Cost: 213.934631
Epoch    6/20 Cost: 89.052269
Epoch    7/20 Cost: 42.876003
Epoch    8/20 Cost: 25.799639
Epoch    9/20 Cost: 19.482420
Epoch   10/20 Cost: 17.143093
Epoch   11/20 Cost: 16.274509
Epoch   12/20 Cost: 15.949721
Epoch   13/20 Cost: 15.825986
Epoch   14/20 Cost: 15.776566
Epoch   15/20 Cost: 15.754660
Epoch   16/20 Cost: 15.742918
Epoch   17/20 Cost: 15.734917
Epoch   18/20 Cost: 15.728307
Epoch   19/20 Cost: 15.722219
Epoch   20/20 Cost: 15.716357
